In [1]:
training_list1 = ['https://catalog.indianatech.edu/content.php?catoid=17&navoid=635#add-drop',
'https://dining.indianatech.edu/',
'https://dining.indianatech.edu/policies/',
'https://dining.indianatech.edu/areas-hours/',
'https://academics.indianatech.edu/programs/diversity-studies-certificate/',
'https://www.indianatech.edu/wellness-clinic/counseling/',
'https://www.indianatech.edu/wellness-clinic/crisis/',
'https://housing.indianatech.edu/',
'https://academics.indianatech.edu/academic-affairs/accreditation/',
'https://academics.indianatech.edu/academic-affairs/irb/',
'https://academics.indianatech.edu/colleges/arts-and-sciences/',
'https://academics.indianatech.edu/programs/forensic-science-bs/',
'https://admissions.indianatech.edu/promiseit/',
'https://admissions.indianatech.edu/freshman-students/',
'https://admissions.indianatech.edu/transfer/',
'https://academics.indianatech.edu/programs/accounting-certificate/',
'https://studentlife.indianatech.edu/get-involved/intramurals/policies/',
'https://studentlife.indianatech.edu/get-involved/student-activities/',
'https://www.indianatech.edu/homecoming-2024/',
'https://online.indianatech.edu/support/locations/',
'https://careercenter.indianatech.edu/internships/',
'https://careercenter.indianatech.edu/students/traditional-undergraduate-services/',
'https://careercenter.indianatech.edu/students/military-services/',
'https://phd.indianatech.edu/program/dissertation/',
'https://phd.indianatech.edu/program/travel/',
'https://academics.indianatech.edu/pathways/certificates/',
'https://registrar.indianatech.edu/calendars/undergrad/',
'https://registrar.indianatech.edu/current-students/resources/enrollment-verifications/',
'https://international.indianatech.edu/graduate/admissions-requirements/',
'https://international.indianatech.edu/visa-immigration/',
'https://alumni.indianatech.edu/events/',
'https://safety.indianatech.edu/',
'https://safety.indianatech.edu/crisis-management-plan/',
'https://safety.indianatech.edu/crisis-management-plan/shelter-secure-and-lockdown-situations-how-to-respond/']

In [2]:
import pandas as pd 
from data.models import WebPageDocumentNew
from settings.settings import Settings
df1 = pd.read_csv('/Users/marynelson/GAI_Academic_Advising_Assistant_MVP/app/evaluation_metrics/itech_processed.csv')
import math 

settings = Settings()
from mongoengine import connect, disconnect
import mongoengine.connection


db_name = settings.MONGO_DB
db_conn = settings.MONGO_CONN_STR

_connect = connect(db_name, host=db_conn)

documents = WebPageDocumentNew.objects.all()

documents_list = []
for doc in documents:
    documents_list.append(doc._data)

print(len(documents_list))


609


In [ ]:
df = pd.DataFrame(documents_list)

# Select numeric columns
numeric_columns = [
    'days_since_modified', 'sentence_count', 'link_depth'
]
numeric_df = df[numeric_columns]

df.head()


In [ ]:
import matplotlib.pyplot as plt
# Plot histogram for the 'score' column
dff = df[df['sentence_count']<5]
dff['sentence_count'].hist(bins=10, edgecolor='black')

# Add title and labels
plt.title('Distribution of Scores')
plt.xlabel('Score')
plt.ylabel('Frequency')

dff.value_counts('sentence_count')

: 

In [ ]:
# Remove duplicate rows based on column 'A'
df_no_duplicates = df.drop_duplicates(subset=['source'])

len(df_no_duplicates)

In [ ]:
dff = dff.sort_values(by='sentence_count', ascending=True)

dff['page_content'].to_list()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalize the numeric features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(numeric_df)

In [ ]:
normalized_features[:5]

In [ ]:
def is_training(url):
    for item in training_list1:
        if url == item:
            return 1
    return 0

# Add a 'label' field to each document
df['label'] = df['source'].apply(is_training)

x = normalized_features
y= df['label']

y.head()

df.value_counts('label')
    

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
# Split into training and testing sets
X_train, X_test, y_train, y_test, train_sources, test_sources = train_test_split(
    x, y, df['source'], test_size=.2, random_state=42, stratify=y
)


In [ ]:

print("Label Distribution in Training Set:")
print(pd.Series(y_train).value_counts())

print("Label Distribution in Testing Set:")
print(pd.Series(y_test).value_counts())

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

print(f'Accuracy: {accuracy:.2f}')
print(f'ROC AUC: {roc_auc:.2f}')

# Check predictions in the entire dataset
df['predicted_prob_all'] = model.predict_proba(x)[:, 1]
df['predicted_label_all'] = model.predict(x)

print("Predicted Label Distribution in Entire Dataset:")
print(df['predicted_label_all'].value_counts())

In [ ]:

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:

# Predict on test set
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]


In [ ]:
#predict all
df['predicted_prob_all'] = model.predict_proba(normalized_features)[:,1]
df['predicted_label_all'] = model.predict(normalized_features)
df.head()



In [ ]:
df.value_counts('predicted_label_all')

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

print(f'Accuracy: {accuracy}')
print(f'ROC AUC: {roc_auc}')

In [ ]:
#include source information in the evaluation output
evaluation_df = pd.DataFrame({
    'source': test_sources,
    'predicted_label': y_pred,
    'predicted_prob': y_pred_prob,
    'actual_label': y_test
})

# Sort the DataFrame by 'predicted_label' in descending order
sorted_df = evaluation_df.sort_values(by='predicted_label', ascending=False)

# Optionally, reset the index
sorted_df = sorted_df.reset_index(drop=True)

# Display the sorted DataFrame
sorted_df.head()



In [ ]:
sorted_df.value_counts('predicted_label')